# 🔧 Environment Setup for SD v1.5 Fine-tuning

環境設置與依賴修復專用 Notebook

## 🎯 用途
- 專門用於環境設置與依賴修復
- 解決 Google Colab 中的套件衝突
- 驗證安裝狀態
- 為主要訓練做準備

## 📋 使用方法
1. 按順序執行所有單元格
2. 如果遇到錯誤，重新啟動運行時
3. 重新執行此 Notebook
4. 確認所有檢查通過後，執行主要訓練 Notebook

## 🔍 步驟 1: 環境檢查

In [ ]:
# 檢查運行環境
import sys
import os
import subprocess

print("🔍 環境檢查中...")
print(f"Python 版本: {sys.version}")
print(f"當前目錄: {os.getcwd()}")

# 檢查是否在 Colab 中
IN_COLAB = 'google.colab' in sys.modules
print(f"運行環境: {'🌐 Google Colab' if IN_COLAB else '💻 Local Environment'}")

if not IN_COLAB:
    print("⚠️ 此 Notebook 專為 Google Colab 設計")
    print("如果在本地運行，請確保已安裝相關依賴")

# 檢查 GPU
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("\n✅ GPU 已啟用")
        # 解析 GPU 信息
        lines = result.stdout.split('\n')
        for line in lines:
            if 'Tesla' in line or 'GeForce' in line or 'Quadro' in line:
                print(f"   GPU: {line.strip()}")
                break
    else:
        print("\n❌ GPU 未啟用")
        print("請啟用 GPU 運行時:")
        print("Runtime > Change runtime type > Hardware accelerator > GPU")
except Exception as e:
    print(f"\n⚠️ 無法檢查 GPU 狀態: {e}")

print("\n📊 系統資源:")
try:
    import psutil
    print(f"   RAM: {psutil.virtual_memory().total / 1024**3:.1f} GB")
    print(f"   CPU: {psutil.cpu_count()} 核心")
except:
    print("   無法獲取系統資源信息")

## 🗑️ 步驟 2: 清理舊依賴

In [ ]:
# 清理可能衝突的套件
print("🗑️ 清理舊版本套件...")
print("這可能需要幾分鐘時間...")

conflicting_packages = [
    "sentence-transformers",
    "transformers", 
    "torch",
    "torchvision",
    "torchaudio",
    "fastai",
    "torchtext",
    "torchdata"
]

for package in conflicting_packages:
    try:
        print(f"   🗑️ 卸載 {package}...")
        !pip uninstall -y {package} -q
        print(f"   ✅ {package} 已卸載")
    except Exception as e:
        print(f"   ⚠️ {package} 卸載失敗: {e}")

# 清理 pip 緩存
print("\n🧹 清理 pip 緩存...")
!pip cache purge

print("\n✅ 清理完成")

## 📦 步驟 3: 安裝 PyTorch

In [ ]:
# 安裝 PyTorch 生態系統
print("📦 安裝 PyTorch 生態系統...")
print("這可能需要 5-10 分鐘...")

# 使用 CUDA 11.8 的穩定版本
!pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0+cu118 \
    --index-url https://download.pytorch.org/whl/cu118 --force-reinstall

# 驗證 PyTorch 安裝
print("\n🔍 驗證 PyTorch 安裝...")
try:
    import torch
    import torchvision
    import torchaudio
    
    print(f"✅ torch: {torch.__version__}")
    print(f"✅ torchvision: {torchvision.__version__}")
    print(f"✅ torchaudio: {torchaudio.__version__}")
    
    if torch.cuda.is_available():
        print(f"✅ CUDA 可用: {torch.version.cuda}")
        print(f"✅ GPU 設備: {torch.cuda.get_device_name(0)}")
        print(f"✅ GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    else:
        print("❌ CUDA 不可用")
        
except ImportError as e:
    print(f"❌ PyTorch 導入失敗: {e}")
    print("請重新啟動運行時並重新執行")

## 📚 步驟 4: 安裝 Transformers

In [ ]:
# 安裝 Transformers
print("📚 安裝 Transformers...")

# 使用特定版本以避免衝突
!pip install "transformers>=4.41.0,<5.0.0" --force-reinstall

# 驗證 Transformers 安裝
print("\n🔍 驗證 Transformers 安裝...")
try:
    import transformers
    print(f"✅ transformers: {transformers.__version__}")
    
    # 測試基本功能
    from transformers import AutoTokenizer
    print("✅ 基本功能測試通過")
    
except ImportError as e:
    print(f"❌ Transformers 導入失敗: {e}")
except Exception as e:
    print(f"❌ Transformers 測試失敗: {e}")

## 🎨 步驟 5: 安裝 Diffusers

In [ ]:
# 安裝 Diffusers 和相關套件
print("🎨 安裝 Diffusers 和相關套件...")

core_packages = [
    "diffusers[torch]",
    "accelerate",
    "peft>=0.4.0",
    "packaging",
    "safetensors",
    "scipy"
]

for package in core_packages:
    try:
        print(f"   📦 安裝 {package}...")
        !pip install {package}
        print(f"   ✅ {package} 安裝完成")
    except Exception as e:
        print(f"   ❌ {package} 安裝失敗: {e}")

# 驗證 Diffusers 安裝
print("\n🔍 驗證 Diffusers 安裝...")
try:
    import diffusers
    from diffusers import StableDiffusionPipeline
    print(f"✅ diffusers: {diffusers.__version__}")
    
    import accelerate
    print(f"✅ accelerate: {accelerate.__version__}")
    
    from peft import LoraConfig
    print(f"✅ peft: 可用")
    
except ImportError as e:
    print(f"❌ Diffusers 相關套件導入失敗: {e}")
except Exception as e:
    print(f"❌ Diffusers 測試失敗: {e}")

## 🧮 步驟 6: 安裝輔助套件

In [ ]:
# 安裝輔助套件
print("🧮 安裝輔助套件...")

utility_packages = [
    "matplotlib",
    "seaborn",
    "numpy",
    "pillow",
    "scikit-learn",
    "tqdm",
    "wandb"
]

for package in utility_packages:
    try:
        print(f"   📦 安裝 {package}...")
        !pip install {package} -q
        print(f"   ✅ {package}")
    except Exception as e:
        print(f"   ❌ {package} 安裝失敗: {e}")

# 最後安裝 sentence-transformers
print("\n🔤 安裝 sentence-transformers...")
try:
    !pip install sentence-transformers
    print("✅ sentence-transformers 安裝完成")
except Exception as e:
    print(f"❌ sentence-transformers 安裝失敗: {e}")
    print("⚠️ 這是可選套件，不影響主要功能")

## ⚡ 步驟 7: 安裝 xformers (可選)

In [ ]:
# 安裝 xformers 以提升性能
print("⚡ 安裝 xformers (可選)...")
print("xformers 可以顯著提升訓練速度")

try:
    !pip install xformers==0.0.22.post7 --index-url https://download.pytorch.org/whl/cu118
    
    # 驗證 xformers
    import xformers
    print(f"✅ xformers: {xformers.__version__}")
    print("✅ xformers 安裝成功，將使用優化的注意力機制")
    
except Exception as e:
    print(f"❌ xformers 安裝失敗: {e}")
    print("⚠️ 將使用標準注意力機制，性能可能較慢但仍可正常運行")

## 🔍 步驟 8: 最終驗證

In [ ]:
# 最終驗證所有套件
print("🔍 最終驗證所有套件...")
print("=" * 50)

# 關鍵套件列表
key_packages = {
    'torch': 'PyTorch',
    'transformers': 'Transformers',
    'diffusers': 'Diffusers',
    'accelerate': 'Accelerate',
    'peft': 'PEFT',
    'numpy': 'NumPy',
    'PIL': 'Pillow',
    'matplotlib': 'Matplotlib'
}

success_count = 0
total_count = len(key_packages)

for package_name, display_name in key_packages.items():
    try:
        module = __import__(package_name)
        if hasattr(module, '__version__'):
            version = module.__version__
        else:
            version = '未知版本'
        print(f"✅ {display_name}: {version}")
        success_count += 1
    except ImportError:
        print(f"❌ {display_name}: 未安裝")
    except Exception as e:
        print(f"⚠️ {display_name}: 錯誤 - {e}")

print("\n" + "=" * 50)
print(f"📊 安裝狀態: {success_count}/{total_count} 套件成功")

# 特殊檢查
print("\n🔍 特殊功能檢查:")

# 檢查 CUDA
try:
    import torch
    if torch.cuda.is_available():
        print(f"✅ CUDA: 可用 (GPU: {torch.cuda.get_device_name(0)})")
    else:
        print("❌ CUDA: 不可用")
except:
    print("❌ CUDA: 檢查失敗")

# 檢查 Stable Diffusion 支援
try:
    from diffusers import StableDiffusionPipeline
    print("✅ Stable Diffusion: 支援")
except:
    print("❌ Stable Diffusion: 不支援")

# 檢查 LoRA 支援
try:
    from peft import LoraConfig
    print("✅ LoRA: 支援")
except:
    print("❌ LoRA: 不支援")

# 檢查 xformers
try:
    import xformers
    print("✅ xformers: 可用")
except:
    print("⚠️ xformers: 不可用 (可選)")

print("\n" + "=" * 50)

if success_count >= total_count - 1:  # 允許一個套件失敗
    print("🎉 環境設置成功！")
    print("✅ 可以開始執行主要訓練 Notebook")
    print("\n💡 建議下一步:")
    print("1. 使用 'SD_v1.5_Complete_Training.ipynb' 進行完整訓練")
    print("2. 或使用 'Quick_Test.ipynb' 進行快速測試")
else:
    print("❌ 環境設置未完成")
    print("🔧 建議解決方案:")
    print("1. 重新啟動運行時 (Runtime > Restart runtime)")
    print("2. 重新執行此 Notebook")
    print("3. 如果問題持續，請檢查錯誤信息")

## 📋 完成檢查清單

### ✅ 必要條件
- [ ] PyTorch 已安裝並可用
- [ ] Transformers 已安裝並可用
- [ ] Diffusers 已安裝並可用
- [ ] CUDA 可用 (GPU 已啟用)
- [ ] PEFT (LoRA) 支援可用

### ⚠️ 可選項目
- [ ] xformers 已安裝 (性能優化)
- [ ] sentence-transformers 已安裝 (特徵提取)
- [ ] wandb 已安裝 (實驗追蹤)

### 🚀 準備就緒
如果上述必要條件都滿足，您可以開始使用主要訓練 Notebook。

### 📞 需要幫助？
如果遇到問題，請:
1. 檢查錯誤訊息
2. 重新啟動運行時
3. 重新執行此 Notebook
4. 查看故障排除指南